In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_generation.load_graphs_and_create_metadata import dataset_metadata
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
import scipy.io as sco
import slam.io as sio
from scipy.special import softmax
import pickle
from scipy.stats import betabinom
import seaborn as sns
import tools.graph_processing as gp
import tools.graph_visu as gv
from matplotlib.pyplot import figure
import pickle
import pandas as pd
from torch_geometric.utils.convert import from_networkx
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch.nn.functional import one_hot
from sklearn.preprocessing import OneHotEncoder
from torch.nn import Linear
import torch.nn.functional as F
import torch_geometric as pyg
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import TopKPooling
from torch_geometric.data import Data
%matplotlib inline

In [2]:
file_template_mesh = '../data/template_mesh/lh.OASIS_testGrp_average_inflated.gii'
file_mesh = '../data/example_individual_OASIS_0061/lh.white.gii'
file_basins = '../data/example_individual_OASIS_0061/alpha0.03_an0_dn20_r1.5_L_area50FilteredTexture.gii'
file_graph = '../data/example_individual_OASIS_0061/OAS1_0061_lh_pitgraph.gpickle'

path_to_labelled_graphs = '../data/Oasis_original_new_with_dummy/labelled_graphs/'

list_graphs = gp.load_graphs_in_list(path_to_labelled_graphs)

attn_mat_all = pickle.load(open( "attn_mat_list.pickle", "rb" ))

In [3]:
for i,g in enumerate(list_graphs):

    attn_mat = attn_mat_all[i]
    edge_attn_dict = {}
    
    for edge in g.edges:
        
        edge_attn_dict[edge] = {"attn_weight":attn_mat[edge[0],edge[1]]}
    
    nx.set_edge_attributes(g, edge_attn_dict)

In [4]:
# graph = list_graphs[0]

# template_mesh = gv.reg_mesh(sio.load_mesh(file_template_mesh))

# # plot the mesh with basin texture
# vb_sc3 = gv.visbrain_plot(template_mesh, caption='Visu on template mesh')
# # process the graph
# # 1 remove potential dummy nodes (which are not connected by any edge and have no coordinate)
# gp.remove_dummy_nodes(graph)
# # 2 compute nodes coordinates in 3D by retrieving the mesh vertex corresponding to each graph node, based on the
# # corresponding node attribute
# nodes_coords = gp.graph_nodes_to_coords(graph, 'ico100_7_vertex_index', template_mesh)
# # 3 eventually compute a mask for masking some part of the graph
# #mask_slice_coord = -15
# #nodes_mask = nodes_coords[:, 2] > mask_slice_coord
# # 4 create the objects for visualization of the graph and add these to the figure
# s_obj3, c_obj3, node_cb_obj3 = gv.show_graph(graph, nodes_coords, node_color_attribute='basin_thickness',
#                                             edge_color_attribute='attn_weight')

# vb_sc3.add_to_subplot(c_obj3)
# vb_sc3.add_to_subplot(s_obj3)
# visb_sc_shape = gv.get_visb_sc_shape(vb_sc3)
# vb_sc3.add_to_subplot(node_cb_obj3, row=visb_sc_shape[0] - 1, col=visb_sc_shape[1] + 1, width_max=200)

# # show the plot on the screen
# vb_sc3.preview()

In [5]:
average_attn = {}

for g in list_graphs:
    
    gp.remove_dummy_nodes(g)
    
    for node in g.nodes:
        
        avg_attn_node = []
        
        for edge in g.edges(node):
            avg_attn_node.append(g.get_edge_data(edge[0],edge[1])['attn_weight'])
            
        average_attn[node] = {'average_attn':np.mean(avg_attn_node)}
        
    nx.set_node_attributes(g, average_attn)

In [ ]:
# # Remove edges based on attention threshold

# deg_attn = {}
# for g in list_graphs:    
#     for edge in g.edges:
        
#         if nx.get_edge_attributes(g,'attn_weight')[edge] < 0.5: # threshold attn_weights             
#             g.remove_edge(edge[0],edge[1])
            
#     deg_attn = dict(g.degree)
            
#     nx.set_node_attributes(g, deg_attn,'deg_attn')

In [ ]:
#list(dict(list_graphs[0].degree).values())

In [ ]:
# all_degree = []
# for g in list_graphs:
#     all_degree.extend(list(dict(g.degree).values()))

In [10]:
all_avg_attn = []
for g in list_graphs:
    all_avg_attn.extend(list(nx.get_node_attributes(g,'average_attn').values()))